In [21]:
import os
import ffmpeg
from datetime import timedelta
import yaml
import pandas as pd
import numpy as np
from itertools import combinations
from itertools import product

In [23]:
df = pd.read_csv('D:\\GITHUB\\Data_CEP\\Exploration\\dataframe_expanded.csv')

In [24]:
df_shot = df[df['primary_type'] == 'shot']

In [25]:
#Attribute Succesfulness:
print(df_shot['on_target'].value_counts())

False    4373
True     2215
Name: on_target, dtype: int64


In [28]:
df_shot['on_target_true'] = 0
df_shot['on_target_false'] = 0

df_shot.loc[(df_shot['on_target'] == True), 'on_target_true'] = 1
df_shot.loc[(df_shot['on_target'] == False), 'on_target_false'] = 1

C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_20640\225499943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shot['on_target_true'] = 0
C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_20640\225499943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shot['on_target_false'] = 0


In [29]:
# Attribute Goal:
print(df_shot['goal'].value_counts())

0    5939
1     649
Name: goal, dtype: int64


In [31]:
df_shot['goal_true'] = 0
df_shot['goal_false'] = 0

df_shot.loc[(df_shot['goal'] == 1), 'goal_true'] = 1
df_shot.loc[(df_shot['goal'] == 0), 'goal_false'] = 1


C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_20640\3423624672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shot['goal_true'] = 0
C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_20640\3423624672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shot['goal_false'] = 0


In [32]:
#Attribute body_part:
print(df_shot['shot_body_part'].value_counts())

right_foot       3124
left_foot        2338
head_or_other    1126
Name: shot_body_part, dtype: int64


In [43]:
df_shot['right_foot_true'] = 0
df_shot['left_foot_true'] = 0
df_shot['head_or_other_true'] = 0

df_shot.loc[(df_shot['shot_body_part'] == 'right_foot'), 'right_foot_true'] = 1
df_shot.loc[(df_shot['shot_body_part'] == 'left_foot'), 'left_foot_true'] = 1
df_shot.loc[(df_shot['shot_body_part'] == 'head_or_other'), 'head_or_other_true'] = 1

C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_20640\3682998674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shot['right_foot_true'] = 0
C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_20640\3682998674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shot['left_foot_true'] = 0
C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_20640\3682998674.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [44]:
print(len(df_shot[df_shot['head_or_other_foot_true'] == 1]))

1126


In [34]:
#Attribute: goal_zone --> not enough values for the different goal zones so excluded from the classification
print(df_shot['goal_zone'].value_counts())

bc     1772
otr     550
otl     528
gb      492
olb     426
gc      407
obr     403
gbr     336
glb     320
ot      278
gl      214
gr      167
gt      156
or      141
ol      139
gtl      69
gtr      52
pbr      29
ptr      22
ptl      22
pr       20
pt       19
plb      14
pl       12
Name: goal_zone, dtype: int64


In [46]:
def select_minimum_clips(df, primary_type, attribute_sets, max_difference_per_set=500, min_clips_per_attribute=1050, chunk_size=10000):
    # Filter the DataFrame for the specified primary type
    primary_df = df 
    # Dictionary to hold the selected clips for each attribute set
    selected_clips_dict = {attr_set: [] for attr_set in attribute_sets}
    
    # Function to check if all attribute sets have enough clips
    def all_attribute_sets_satisfied():
        return all(len(clips) >= min_clips_per_attribute * len(attr_set) for attr_set, clips in selected_clips_dict.items())
    
    # Iterate through the DataFrame once per attribute set
    for attr_set in attribute_sets:
        attr_clips = []
        for attr in attr_set:
            # Extract clips where the specific column (attr) is True (1)
            attr_clips.extend(primary_df[primary_df[attr] == 1]['event_id'].tolist())
        
        # Shuffle the clips to randomize selection
        np.random.shuffle(attr_clips)
        
        # Select exactly min_clips_per_attribute clips for this attribute set
        selected_clips_dict[attr_set] = attr_clips[:min_clips_per_attribute]
        
        # Check if all attribute sets have enough clips after each set
        if all_attribute_sets_satisfied():
            break
    
    # Combine all selected clips ensuring no duplicates
    selected_clips = set()
    for clips in selected_clips_dict.values():
        selected_clips.update(clips)

    return selected_clips

In [47]:
df_shuffled = df_shot.sample(frac=1).reset_index(drop=True)
primary_type = 'shot'
attribute_sets = [('right_foot_true', 'left_foot_true', 'head_or_other_true'),
    ('on_target_true', 'on_target_false'),
    ('goal_true', 'goal_false')
]

# Select minimum clips to cover all classifications
selected_clips = select_minimum_clips(df_shuffled, primary_type, attribute_sets)
print(f"Selected clip IDs: {selected_clips}")
print(f"Number of selected clips: {len(selected_clips)}")

Selected clip IDs: {1747542016, 1797029888, 1737695238, 2108342278, 1731018760, 2090598409, 1781702667, 1772937230, 2121408530, 2108342291, 2010144787, 1777958935, 1770815514, 1866522655, 1866522658, 1764769827, 1871880232, 1838653481, 1789812778, 1773518892, 1720901681, 1789812789, 1937457207, 1768202297, 1850122305, 2089508932, 1759887434, 1767071820, 1810759759, 2089508944, 2099060817, 1777958999, 2126766170, 1759887451, 1821614172, 2097700958, 2090598495, 2090598503, 1813545065, 1759887469, 2095595630, 1838653552, 2121408631, 1874452599, 1819115643, 1777959037, 2095595658, 1874329739, 1724481681, 1832231058, 1724481682, 2095595668, 1723220116, 1884577942, 1764769943, 1886314644, 1819115674, 1770815644, 1886314652, 2124456094, 1937457318, 1853472935, 2121408678, 1742405802, 2093875372, 2099060909, 1789812910, 1742586030, 2114314416, 2099060915, 2121408697, 2100199610, 1731018941, 1731018943, 1791180996, 1808261319, 1819115720, 1832460490, 2115813579, 1832231127, 1731018975, 21309032

In [48]:
df_selected_clips = df_shuffled[df_shuffled['event_id'].isin(selected_clips)].reset_index(drop=True)

In [50]:
df_selected_clips.to_csv('D:\\GITHUB\\Data_CEP\\Exploration\\selected_clips_shot.csv', index=False)

In [49]:
list = ['right_foot_true', 'left_foot_true', 'head_or_other_true', 'on_target_true', 'on_target_false','goal_true', 'goal_false']
for att in list:
    print(len(df_selected_clips[df_selected_clips[att] == 1]))

1274
938
448
993
2406
254
2406
